In [1]:
import torch
import cv2
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import Any
from matplotlib.patches import Rectangle
from PIL import Image
import imageio
import sys
sys.path.append(os.path.dirname(os.getcwd()))

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms import v2

from src.models.unet import BaseUNet

In [27]:
img_list  = sorted(glob.glob(os.path.join("/home/syurtseven/gsoc-2024/data/stomach_masks", '*image*.png')))
mask_list  = sorted(glob.glob(os.path.join("/home/syurtseven/gsoc-2024/data/stomach_masks", '*mask*.png')))

In [36]:
len(img_list)

8627

In [28]:
def get_cases(img_list, n_cases=10):
    case_set = set()
    for img in img_list:
        case_num = img.split('/')[-1].split('_')[0]
        case_set.add(case_num)

    val_cases = list(case_set)[:10]
    return val_cases

In [30]:
def split_data(val_cases):

    training   = []
    validation = []

    for i in range(len(img_list)):

        case_num = img_list[i].split('/')[-1].split('_')[0]

        if case_num in val_cases:
            validation.append(img_list[i])
            validation.append(mask_list[i])
        
        else:
            training.append(img_list[i])
            training.append(mask_list[i])


    return training, validation


In [31]:
val_cases = get_cases(img_list)
training, validation = split_data(val_cases)

In [38]:
print(len(training)//2)
print(len(validation)//2)

7508
1119


In [35]:
validation

['/home/syurtseven/gsoc-2024/data/stomach_masks/case119_day0_slice_0056_image.png',
 '/home/syurtseven/gsoc-2024/data/stomach_masks/case119_day0_slice_0056_mask.png',
 '/home/syurtseven/gsoc-2024/data/stomach_masks/case119_day0_slice_0057_image.png',
 '/home/syurtseven/gsoc-2024/data/stomach_masks/case119_day0_slice_0057_mask.png',
 '/home/syurtseven/gsoc-2024/data/stomach_masks/case119_day0_slice_0058_image.png',
 '/home/syurtseven/gsoc-2024/data/stomach_masks/case119_day0_slice_0058_mask.png',
 '/home/syurtseven/gsoc-2024/data/stomach_masks/case119_day0_slice_0059_image.png',
 '/home/syurtseven/gsoc-2024/data/stomach_masks/case119_day0_slice_0059_mask.png',
 '/home/syurtseven/gsoc-2024/data/stomach_masks/case119_day0_slice_0060_image.png',
 '/home/syurtseven/gsoc-2024/data/stomach_masks/case119_day0_slice_0060_mask.png',
 '/home/syurtseven/gsoc-2024/data/stomach_masks/case119_day0_slice_0061_image.png',
 '/home/syurtseven/gsoc-2024/data/stomach_masks/case119_day0_slice_0061_mask.png'

In [5]:
with open('/home/syurtseven/gsoc-2024/reports/size/0/all_image_paths.txt',"r") as f:
    files_str = f.read()

In [15]:
[mask.replace('image', 'mask') + '.png' for mask in files_str.split('.png')]

['/home/syurtseven/gsoc-2024/data/stomach_masks/case122_day0_slice_0062_mask.png',
 '/home/syurtseven/gsoc-2024/data/stomach_masks/case54_day0_slice_0049_mask.png',
 '/home/syurtseven/gsoc-2024/data/stomach_masks/case142_day16_slice_0067_mask.png',
 '/home/syurtseven/gsoc-2024/data/stomach_masks/case40_day0_slice_0074_mask.png',
 '/home/syurtseven/gsoc-2024/data/stomach_masks/case114_day15_slice_0069_mask.png',
 '/home/syurtseven/gsoc-2024/data/stomach_masks/case58_day0_slice_0056_mask.png',
 '/home/syurtseven/gsoc-2024/data/stomach_masks/case149_day12_slice_0076_mask.png',
 '/home/syurtseven/gsoc-2024/data/stomach_masks/case66_day34_slice_0093_mask.png',
 '/home/syurtseven/gsoc-2024/data/stomach_masks/case34_day0_slice_0047_mask.png',
 '/home/syurtseven/gsoc-2024/data/stomach_masks/case119_day21_slice_0065_mask.png',
 '/home/syurtseven/gsoc-2024/data/stomach_masks/case138_day12_slice_0098_mask.png',
 '/home/syurtseven/gsoc-2024/data/stomach_masks/case78_day0_slice_0098_mask.png',
 '/h

In [12]:
images_path = [image+'.png' for image in files_str.split('.png')]

In [8]:
device = 'cuda:0'
load_path = '/home/syurtseven/gsoc-2024/reports/seg/2/model/model_20.pt'
model  = BaseUNet(in_channels=1, out_channels=1)
model.load_state_dict(torch.load(load_path))
model  = model.to(device)